In [1]:
#%matplotlib inline
import sys
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/')
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/weather_mast/')
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/MEPS/')
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/Retrieval_MEPS/')
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import datetime
import math

import createFolder as cF
import calc_date as cd
import plot_sfc_spaghetti_ret as spagh
import save_fig as SF
import get_Haukeli_obs_data as obsDat
import calc_48h_acc as acc
import fill_values as fv
import plot_vertical as pvert

import os

import pandas as pd
import matplotlib as mpl
mpl.style.use('ggplot')

In [2]:
year = 2016
month = 12
tid = '00'
#tid = '18'
station = 'Haukeliseter'
savefig = 1 # 1 = yes, 0 = no



In [3]:
#### days 
#t = ['17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27']
t = ['20', '21', '22', '23', '24', '25', '26', '27']
#t = ['22']

In [4]:
var_name_sfc = 'surface_air_pressure'
var_name_2m = 'air_temperature_2m'
var_name_10m_u = 'x_wind_10m'
var_name_10m_v = 'y_wind_10m'

unit = '[Pa]'
figdir_sfc = '../../Figures/MEPS/%s' %(var_name_sfc)
figdir_2m = '../../Figures/MEPS/%s' %(var_name_2m)
figdir_10m = '../../Figures/MEPS/wind'
figdir_scatter='../../Figures/MEPS/scatter_MEPS_obs'

# Folder where the figures are being saved    
cF.createFolder('%s/' %(figdir_sfc))
cF.createFolder('%s/' %(figdir_2m))
cF.createFolder('%s/' %(figdir_10m))
cF.createFolder('%s/' %(figdir_scatter))
form = 'png'

In [5]:
level = 'sfc'
### MEPS
ncdir_sfc = '../../Data/MEPS/%s/%s_%s' %(station,level,tid)
level = '2m'
ncdir_2m = '../../Data/MEPS/%s/%s_%s' %(station,level,tid)

level = '10m'
ncdir_10m = '../../Data/MEPS/%s/%s_%s' %(station,level,tid)


################################################################
### Haukeli obs.
txtdir = '../../Data/eklima/'
txt_filename = 'Observations-hour_minutes'

################################################################
### Retrieval 
nc_dir_retrieval = '../../Data/Retrieved_SWC'

################################################################



In [6]:
# read in the Haukeliseter observation file, sorted by daily values
Haukeli = pd.read_csv('%s/%s.txt' %(txtdir, txt_filename),\
                    sep = ';',header=19, skipfooter = 11, engine= 'python')

DateHour = Haukeli['Date-Hour(UTC)']

TA = Haukeli['TA'].astype(float)           # Air temperature;degC --> Lufttemperatur ved observasjonstiden i 2 m høyde
DD   = Haukeli['DD'].astype(float)             # Wind direction (FF); [degrees]
FF   = Haukeli['FF'].astype(float)             # Wind speed (10 meters above ground);m/s

TA0, dt_TA0, hour_TA0, day_TA0, month_TA0, year_TA0 = obsDat.valid_values(TA, DateHour)
dt, Temp, hour, dy, mm, yr = obsDat.arange_daily(TA, DateHour)

   

In [7]:
idxFF = np.where(FF[:] == -9999)
idxDD = np.where(DD[:] == -9999)

FF[idxFF[0][:]] = np.nan
DD[idxDD[0][:]] = np.nan

### calculate the U, V wind component for barb plot
# http://colaweb.gmu.edu/dev/clim301/lectures/wind/wind-uv.html

# first calculate the mathematical wind direction in deg
md_deg = 270 - DD
for k in range(0,md_deg.shape[0]):
    if md_deg[k] <0 :
        md_deg[k] = md_deg[k] +360
md_rad = math.pi/180. * md_deg
uwind = FF*np.cos(md_rad)
vwind = FF*np.sin(md_rad)

In [8]:
# read in the Haukeliseter observation file, sorted by daily values
### Pressure
Haukeli2 = pd.read_csv('%s/%s_pressure.txt' %(txtdir, txt_filename),\
                    sep = ';',header=20, skipfooter = 2, 
                       engine= 'python')


PR = Haukeli2['PR'].astype(float)           # PR;Air pressure at sea level;hPa

PR0, dt_PR0, hour_PR0, day_PR0, month_PR0, year_PR0 = obsDat.valid_values(PR, DateHour)
dt, pres, hour, dy, mm, yr = obsDat.arange_daily(PR, DateHour)

In [9]:
FF_wind, DD_wind, dt_wind, hour_wind, day_wind, month_wind, year_wind = obsDat.valid_values_wind(FF, DD, DateHour)

In [10]:
fn_sfc        = dict()
fn_2m         = dict()
fn_10m        = dict()

sfc_pressure = dict()
mslp = dict()
air_temperature_2m = dict()
x_wind_10m    = dict()
y_wind_10m    = dict()

time_sfc      = dict()
time_2m       = dict()
time_10m      = dict()

ini_day       = []
calday        = []
calmon        = []


lead_time_sfc = dict()
lead_time_2m  = dict()
lead_time_10m = dict()

title         = dict()

wd_MEPS    = dict()
ws_MEPS    = dict()

In [11]:
diff_wd = dict()
Diff_wd_0 = []
Diff_wd_1 = []
Diff_wd_2 = []
Diff_wd_3 = []
Diff_wd_4 = []
Diff_wd_5 = []
Diff_wd_6 = []
Diff_wd_7 = []
Diff_wd_8 = []
Diff_wd_9 = []


all_day_max_WD = []
all_day_min_WD = []



In [12]:
diff_ws = dict()
Diff_ws_0 = []
Diff_ws_1 = []
Diff_ws_2 = []
Diff_ws_3 = []
Diff_ws_4 = []
Diff_ws_5 = []
Diff_ws_6 = []
Diff_ws_7 = []
Diff_ws_8 = []
Diff_ws_9 = []


all_day_max_WS = []
all_day_min_WS = []



In [13]:
diff_temp = dict()
Diff_temp_0 = []
Diff_temp_1 = []
Diff_temp_2 = []
Diff_temp_3 = []
Diff_temp_4 = []
Diff_temp_5 = []
Diff_temp_6 = []
Diff_temp_7 = []
Diff_temp_8 = []
Diff_temp_9 = []


all_day_max_temp = []
all_day_min_temp = []



In [14]:
diff_mslp = dict()
Diff_mslp_0 = []
Diff_mslp_1 = []
Diff_mslp_2 = []
Diff_mslp_3 = []
Diff_mslp_4 = []
Diff_mslp_5 = []
Diff_mslp_6 = []
Diff_mslp_7 = []
Diff_mslp_8 = []
Diff_mslp_9 = []


all_day_max_mslp = []
all_day_min_mslp = []



In [15]:
calendar_day = []
initialisation_day = []
calendar_month = []
calendar_year = []
initialisation_hh = []

In [16]:
def plt_variable(lead_time_sfc,wd_MEPS,WD,var):
    
    fig = plt.figure(figsize=(20,7))
    ax = plt.axes()
# Vertical line to show end of day
    ax.axvline(0,color = spagh.vert_col, linewidth = 3)
    ax.axvline(24,color = spagh.vert_col, linewidth = 3)
    ax.axvline(48,color = spagh.vert_col, linewidth = 3)




### ensemble member
    for ens_memb in range(2,10):
        ax.plot(lead_time_sfc[ens_memb],wd_MEPS[ens_memb],color = spagh.memb_col,
           linestyle='-', label='_nolegend_')
    ax.plot(lead_time_sfc[1], wd_MEPS[1], color = spagh.memb_col,
           linestyle = '-', label = 'ensemble member')
    ax.plot(lead_time_sfc[0], wd_MEPS[0], 'k', linewidth = 4, label = 'deterministic')

### observation
    plt.plot(np.arange(0,np.asarray(WD).shape[0]), WD,  markersize=20,  linestyle='--', 
         label = 'observation', linewidth= 4)


### fine tuning
    lgd = ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.37),
          fancybox=True, shadow=True, ncol=3, fontsize=spagh.label_fs)
    frame = lgd.get_frame()
    frame.set_facecolor('white')

# xaxis
    a = lead_time_sfc[0][0:48]
    ax.set_xlim(-0.5,49-0.5)
    ax.set_xlabel('time', fontsize=spagh.label_fs)
    ax.set_xticks(np.arange(0,49,3))
  #  dates = pvert.dates_plt_00(hour, mm, dy, yr, ini_day)
    xdays = ['%s-%s-%s' %(year,month,ini_day), '',
        6,'',12,'', 18,'',
        '%s-%s-%s' %(year,month,ini_day+1), '',
        6,'', 12,'', 18,'',
        '%s-%s-%s' %(year,month,ini_day+2)]
    ax.set_xticklabels(xdays, rotation = 25, fontsize = spagh.tick_fs)
# title
    ax.set_title(title, fontsize=spagh.fontsize, color =spagh.blue )    
    
    if var == 'WD':
        # Horizontal line to show Wind direction
        ax.axhline(90,color=spagh.vert_col, linewidth= 3)
        ax.axhline(180,color=spagh.vert_col, linewidth= 3)
        ax.axhline(270,color=spagh.vert_col, linewidth= 3)
        ax.axhline(360,color=spagh.vert_col, linewidth= 3)
        # yaxis
        ax.set_ylabel('Wind direction', fontsize=spagh.label_fs)
        ax.set_ylim(-0.5,360)
        T = np.arange(0,361,45)
        ax.set_yticks(T)
        ax.set_yticklabels(['N', '', 'E', '','S', '', 'W', '' , 'N'],fontsize = spagh.tick_fs)
    elif var == 'WS':
        # yaxis
        ax.set_ylabel('Wind speed [m$\,$s$^{-1}$]', fontsize = spagh.label_fs)
        ax.set_ylim(0,30)
        ax.set_yticks(np.arange(0,32.5,2.5))
        ax.set_yticklabels([0, '', 5, '',10,'',15,'',20,'',25,'',30], fontsize=spagh.label_fs)
    elif var == 'T2':
        ax.axhline(0,color=spagh.vert_col, linewidth= 3)
        # yaxis
        ax.set_ylabel('Air Temperature [$^\circ$C]', fontsize=spagh.label_fs)
        ax.set_ylim(-9,6)
        T = np.arange(-9,7)
        ax.set_yticks(T)
        ax.set_yticklabels([-9, '' , '', -6, '' , '', -3, '' , '', 0, 
                            '' , '', 3, '' , '', 6], fontsize=spagh.tick_fs)
    elif var == 'SP':
        # yaxis
        ax.set_ylabel('Sea Level Pressure [hPa]', fontsize=spagh.label_fs)
        ax.set_ylim(975, 1040)
        ax.set_yticks(np.arange(975,1045,5))
        ax.set_yticklabels(['' , 980,'', '','', 1000, '','','', 1020, '','','', 1040], fontsize=spagh.tick_fs)
        

# tight layout
    plt.tight_layout()
    
    return(lgd)



In [17]:
def plt_scatter_obs_model(tot, precipitation_amount_acc,colors,var):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
    
#    ax.plot([-0.5, 80], [-0.5, 80.], linestyle='-',color=spagh.memb_col)
    for ens_memb in range(0,1):
        if len(precipitation_amount_acc[ens_memb]) == 0:
            continue
        else:
            ax.scatter(tot,precipitation_amount_acc[ens_memb][:np.asarray(tot).shape[0]],color=colors, 
                   alpha = 0.7, s = 150,
                      label='%s, %s %s %s %s UTC' %(calday,ini_day, 
                                        calmon,year,hh))
    for ens_memb in range(1,2):
        if len(precipitation_amount_acc[ens_memb]) == 0:
            continue
        else:
            ax.scatter(tot,precipitation_amount_acc[ens_memb][:np.asarray(tot).shape[0]],color=colors, 
                   alpha = 0.7, s = 150,
                      label='_nolegend_')
            
    for ens_memb in range(2,10):
        if len(precipitation_amount_acc[ens_memb]) == 0:
            continue
        else:
            ax.scatter(tot[::3],precipitation_amount_acc[ens_memb][:np.asarray(tot).shape[0]],color=colors, 
                   alpha = 0.7, s = 150,
                      label='_nolegend_')
### fine tuning
    lgd = ax.legend(loc='lower right',
         #fancybox=True, shadow=True, #ncol=3, 
          fontsize=spagh.label_fs-8)
    frame = lgd.get_frame()
    frame.set_facecolor('white')
    plt.setp(lgd.get_texts(), color=spagh.blue)
    
    
    
    if var == 'WD':
        ax.plot([0, 360], [0, 360.], linestyle='-',color=spagh.memb_col) 
        # Horizontal line to show Wind direction
        ax.axhline(90,color=spagh.vert_col, linewidth= 3)
        ax.axhline(180,color=spagh.vert_col, linewidth= 3)
        ax.axhline(270,color=spagh.vert_col, linewidth= 3)
        ax.axhline(360,color=spagh.vert_col, linewidth= 3)
        # Vertical line to show Wind direction
        ax.axvline(90,color=spagh.vert_col, linewidth= 3)
        ax.axvline(180,color=spagh.vert_col, linewidth= 3)
        ax.axvline(270,color=spagh.vert_col, linewidth= 3)
        ax.axvline(360,color=spagh.vert_col, linewidth= 3)
        # yaxis
        ax.set_ylim(0,360)
        ax.set_yticks(np.arange(0,361,45)) 
        ax.set_yticklabels(['N', '', 'E', '', 'S','','W','','N'],fontsize = spagh.tick_fs)
        # xaxis
        ax.set_xlim(0,360)
        ax.set_xticks(np.arange(0,361,45))  
        ax.set_xticklabels(['N', '', 'E', '', 'S','','W','','N'], fontsize = spagh.tick_fs)
        # title
        ax.set_title('Wind direction', fontsize=spagh.fontsize) 
    elif var == 'WS':
        ax.plot([0, 30], [0, 30.], linestyle='-',color=spagh.memb_col) 
        # yaxis
        ax.set_ylim(0,30)
        ax.set_yticks(np.arange(0,32.5,2.5))
        ax.set_yticklabels([0, '', 5, '',10,'',15,'',20,'',25,'',30], fontsize=spagh.label_fs)
        # xaxis
        ax.set_xlim(0,30)
        ax.set_xticks(np.arange(0,32.5,2.5))
        ax.set_xticklabels([0, '', 5, '',10,'',15,'',20,'',25,'',30], fontsize=spagh.label_fs)
        # title
        ax.set_title('Wind speed [m$\,$s$^{-1}$]', fontsize=spagh.fontsize) 
    elif var == 'T2':
        ax.plot([-9,6], [-9,6], linestyle='-',color=spagh.memb_col) 
        ax.axhline(0,color=spagh.vert_col, linewidth= 3)
        ax.axvline(0,color=spagh.vert_col, linewidth= 3)
        # yaxis
        ax.set_ylim(-9,6)
        T = np.arange(-9,7)
        ax.set_yticks(T)
        ax.set_yticklabels([-9, '' , '', -6, '' , '', -3, '' , '', 0, 
                            '' , '', 3, '' , '', 6], fontsize=spagh.tick_fs)
        # xaxis
        ax.set_xlim(-9,6)
        ax.set_xticks(T)
        ax.set_xticklabels([-9, '' , '', -6, '' , '', -3, '' , '', 0, 
                            '' , '', 3, '' , '', 6], fontsize=spagh.tick_fs)
        # title
        ax.set_title('Air Temperature [$^\circ$C]', fontsize=spagh.fontsize) 
    elif var == 'SP':
        ax.plot([975,1040], [975,1040], linestyle='-',color=spagh.memb_col) 
        # yaxis
        ax.set_ylim(975, 1040)
        ax.set_yticks(np.arange(975,1045,5))
        ax.set_yticklabels(['' , 980,'', '','', 1000, '','','', 1020, '','','', 1040], fontsize=spagh.tick_fs)
        # xaxis
        ax.set_xlim(975, 1040)
        ax.set_xticks(np.arange(975,1045,5))
        ax.set_xticklabels(['' , 980,'', '','', 1000, '','','', 1020, '','','', 1040], fontsize=spagh.tick_fs)
        # title
        ax.set_title('Sea Level Pressure [hPa]', fontsize=spagh.fontsize)
        
    ax.set_ylabel('MEPS forecast', fontsize=spagh.label_fs)
    ax.set_xlabel('observation', fontsize=spagh.label_fs)


# tight layout
    plt.tight_layout()

In [18]:
def plt_scatter_daily(diff, lead_time_sfc, tot,colors, var):
    
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
# Vertical line to see if too high or too little
    ax.axvline(0, color =spagh.vert_col, linewidth=3)

# Horizontal line to show end of day
    ax.axhline(0,color = spagh.vert_col, linewidth = 3)
    ax.axhline(24,color = spagh.vert_col, linewidth = 3)
    ax.axhline(48,color = spagh.vert_col, linewidth = 3)

    
    for ens_memb in range(0,1):
        ax.scatter(diff[ens_memb],np.arange(0,49),
                   color=colors, alpha = 0.7, s = 150,
                    label='%s, %s %s %s %s UTC' %(calday,ini_day, 
                                        calmon,year,hh)) 
    for ens_memb in range(1,10):
        ax.scatter(diff[ens_memb],np.arange(0,49),
                   color=colors, alpha = 0.7, s = 150)#,
              #label='_nolegend_')
    
### fine tuning
    lgd = ax.legend(loc='lower right',
         #fancybox=True, shadow=True, #ncol=3, 
          fontsize=spagh.label_fs-8)
    frame = lgd.get_frame()
    frame.set_facecolor('white')
    plt.setp(lgd.get_texts(), color=spagh.blue)
    
# yaxis
    a = lead_time_sfc[0][0:48]
    ax.set_ylim(-0.5,49-0.5)
    ax.set_ylabel('forecast time', fontsize=spagh.label_fs)
    ax.set_yticks(np.arange(0,49))
    
    xdays = [0, '', '', 3, '' , '',
        6,'','', 9,'','',12,'','',15,'','',18,'','',
         21, '','',
        24, '','',27, '','', 30, '','',
        33,'','', 36,'','',39,'','',42,'','',45,'','',
        48]
    ax.set_yticklabels(xdays, #rotation = 25, 
                   fontsize = spagh.tick_fs)

    if var == 'WD':
        # Horizontal line to show Wind direction
        ax.axvline(45,color=spagh.vert_col, linewidth= 3)
        ax.axvline(90,color=spagh.vert_col, linewidth= 3)
        ax.axvline(90+45,color=spagh.vert_col, linewidth= 3)
        ax.axvline(180,color=spagh.vert_col, linewidth= 3)
        ax.axvline(-45,color=spagh.vert_col, linewidth= 3)
        ax.axvline(-90,color=spagh.vert_col, linewidth= 3)
        ax.axvline(-90-45,color=spagh.vert_col, linewidth= 3)
        ax.axvline(-180,color=spagh.vert_col, linewidth= 3)
        # xaxis
        ax.set_xlim(-112.5,112.5)
        ax.set_xticks(np.arange(-112.5,135,22.5))  
        ax.set_xticklabels([ '', '-90', '', '-45','','0','','45','','90'], fontsize = spagh.tick_fs)
        # title
        ax.set_title('Wind direction', fontsize=spagh.fontsize) 
        ax.set_xlabel('Difference [$^\circ$]' , fontsize=spagh.label_fs)
    elif var == 'WS':
        # xaxis
        ax.set_xlim(-10,15)
        ax.set_xticks(np.arange(-10,17.5,2.5))
        ax.set_xticklabels([-10,'', -5, '',0, '', 5, '',10,'',15], fontsize=spagh.label_fs)
        # title
        ax.set_title('Wind speed', fontsize=spagh.fontsize) 
        ax.set_xlabel('Difference [m$\,$s$^{-1}$]' , fontsize=spagh.label_fs)
        
    elif var == 'T2':
        # xaxis
        ax.set_xlim(-9,4)
        ax.set_xticks(np.arange(-9,5))
        ax.set_xticklabels([-9, '' , '', -6, '' , '', -3, '' , '', 0, 
                            '' , '', 3, '' , ''], fontsize=spagh.tick_fs)
        # title
        ax.set_title('Air Temperature', fontsize=spagh.fontsize) 
        ax.set_xlabel('Difference [$^\circ$C]' , fontsize=spagh.label_fs)
        
    elif var == 'SP':
        # xaxis
        ax.set_xlim(-8, 8)
        ax.set_xticks(np.arange(-8,9))
        ax.set_xticklabels([-8,'' , -6,'', -4,'',-2,'', 0, '',2,'',4,'',6,'',8], fontsize=spagh.tick_fs)
        # title
        ax.set_title('Sea Level Pressure [hPa]', fontsize=spagh.fontsize)
        
# tight layout
    plt.tight_layout()
    
    
#    return(lgd);

In [19]:
def calc_diff(diff,variable,tot,Difference_0,Difference_1, \
              Difference_2, Difference_3, Difference_4,Difference_5, \
              Difference_6,Difference_7, Difference_8,Difference_9, var):
    #### calcualate the difference between DoFe and ensemble members and save daily values  
    
    if var == 'T2' or var == 'SP':
        for ens_memb in range(0,2):
       # print(ens_memb)
            if len(variable[ens_memb] ) == 0:
                diff[ens_memb] = np.empty(shape=(variable[0][:np.asarray(tot).shape[0],0].shape))
                diff[ens_memb][:] = np.nan
            else:
                diff[ens_memb] = variable[ens_memb][:np.asarray(tot).shape[0],0]-tot
    
        for ens_memb in range(2,10):
        #print(ens_memb)
            if len(variable[ens_memb] ) == 0:
                diff[ens_memb] = np.empty(shape=(variable[0][:np.asarray(tot).shape[0],0].shape))
                diff[ens_memb][:] = np.nan
            else: 
                diff[ens_memb] = variable[ens_memb][:np.asarray(tot).shape[0],0]-tot
            
           
    elif var == 'WD' or var == 'WS':
        for ens_memb in range(0,2):
       # print(ens_memb)
            if len(variable[ens_memb] ) == 0:
                diff[ens_memb] = np.empty(shape=(variable[0][:np.asarray(tot).shape[0]].shape))
                diff[ens_memb][:] = np.nan
            else:
                diff[ens_memb] = variable[ens_memb][:np.asarray(tot).shape[0]]-tot
    
        for ens_memb in range(2,10):
        #print(ens_memb)
            if len(variable[ens_memb] ) == 0:
                diff[ens_memb] = np.empty(shape=(variable[0][:np.asarray(tot).shape[0]].shape))
                diff[ens_memb][:] = np.nan
            else: 
                diff[ens_memb] = variable[ens_memb][:np.asarray(tot).shape[0]]-tot
        
    Difference_0.append(diff[0])
    Difference_1.append(diff[1])
    Difference_2.append(diff[2])
    Difference_3.append(diff[3])
    Difference_4.append(diff[4])
    Difference_5.append(diff[5])
    Difference_6.append(diff[6])
    Difference_7.append(diff[7])
    Difference_8.append(diff[8])
    Difference_9.append(diff[9])
    
    
    
    
    return(diff, Difference_0,Difference_1, Difference_2, Difference_3, Difference_4,Difference_5, Difference_6,Difference_7, Difference_8,Difference_9);#,all_day_max, all_day_min);



def diff_all_day(diff, all_day_max, all_day_min):
    day_diff = []
    for ens_memb in range(0,10):
        day_diff.append(diff[ens_memb])
    max_diff = []
    min_diff = []
    for i in range(0,np.asarray(day_diff).shape[1],3):
        max_diff.append(np.nanmax(np.asarray(day_diff)[:,i]))
        min_diff.append(np.nanmin(np.asarray(day_diff)[:,i]))
        
    
    all_day_max.append(max_diff)
    all_day_min.append(min_diff)
    
    return(all_day_max,all_day_min);

In [20]:
champ = 255.
no1 = np.array([79,94,26])/champ
no2 = np.array([131,156,45])/champ
no3 = np.array([71,153,112])/champ
no4 = np.array([77,111,157])/champ
no5 = np.array([157,58,55])/champ
no6 = np.array([211,120,50])/champ
no7 = np.array([218,181,70])/champ

colors = [no1, no2, no3, no4, no5, no6, no7]

In [21]:
#t = ['24']
for day in t:
    ### connect pressure data for two days
    df1 = PR0[int(day)-1]
    df2 = PR0[int(day)]
    Pressure = np.concatenate((df1,df2),axis=0)
    df3 = PR0[int(day)+1][0]
    Pressure = np.append(Pressure,df3)
        
    ### connect temperature data for two days
    df1 = TA0[int(day)-1]
    df2 = TA0[int(day)]
    Temperature = np.concatenate((df1,df2),axis=0)
    df3 = TA0[int(day)+1][0]
    Temperature = np.append(Temperature,df3)
    
    ### connect wind data for two days
    df1 = DD_wind[int(day)-1]
    df2 = DD_wind[int(day)]
    WD = np.concatenate((df1,df2),axis=0)
    df3 = DD_wind[int(day)+1][0]
    WD = np.append(WD,df3)
    
    df1 = FF_wind[int(day)-1]
    df2 = FF_wind[int(day)]
    WS = np.concatenate((df1,df2),axis=0)
    df3 = FF_wind[int(day)+1][0]
    WS = np.append(WS,df3)
    
    
    for ens_memb in range(0,10):
# read in the netcdf surface file
        fn_sfc[ens_memb] = netCDF4.Dataset('%s/%s%s%s_%s_%s.nc' %(ncdir_sfc,year,month,day,tid,ens_memb))



        time_sfc[ens_memb] = fn_sfc[ens_memb].variables['time']
        time_sfc[ens_memb] = fv.fill_nan(time_sfc[ens_memb][:])
# initialisation time for plotting
        ini_day = (datetime.datetime.utcfromtimestamp(time_sfc[0][0]).day)   # day of initialisation
        hh = (datetime.datetime.utcfromtimestamp(time_sfc[0][0]).hour)       # first hour of initialisation
        calday, calmon = cd.get_dayname(year, month, ini_day)
# Read in the surface variable (pressure) 
        sfc_pressure[ens_memb] = fn_sfc[ens_memb].variables['surface_air_pressure'][:]
        sfc_pressure[ens_memb] = sfc_pressure[ens_memb]/100
    
        mslp[ens_memb] = fn_sfc[ens_memb].variables['air_pressure_at_sea_level'][:]
        mslp[ens_memb] = mslp[ens_memb]/100
        
# read in the netcdf 2m temperature file
        fn_2m[ens_memb] = netCDF4.Dataset('%s/%s%s%s_%s_%s.nc' %(ncdir_2m,year,month,day,tid,ens_memb))
    
        time_2m[ens_memb] = fn_2m[ens_memb].variables['time']
        time_2m[ens_memb] = fv.fill_nan(time_2m[ens_memb][:])
# read in the tempearature variable
        air_temperature_2m[ens_memb] = fn_2m[ens_memb].variables['air_temperature_2m'][:]
        air_temperature_2m[ens_memb] = air_temperature_2m[ens_memb]-273.15

# read in the netcdf 10m wind file
        fn_10m[ens_memb] = netCDF4.Dataset('%s/%s%s%s_%s_%s.nc' %(ncdir_10m,year,month,day,tid,ens_memb))
    
        time_10m[ens_memb] = fn_10m[ens_memb].variables['time']
        time_10m[ens_memb] = fv.fill_nan(time_10m[ens_memb][:])
# read in the wind variable
        x_wind_10m[ens_memb] = fn_10m[ens_memb].variables['x_wind_10m'][:]
        y_wind_10m[ens_memb] = fn_10m[ens_memb].variables['y_wind_10m'][:]
    

# lead time
        lead_time_sfc[ens_memb] = np.arange(0,time_sfc[ens_memb].shape[0])
        lead_time_2m[ens_memb] = np.arange(0,time_2m[ens_memb].shape[0])
        lead_time_10m[ens_memb] = np.arange(0,time_10m[ens_memb].shape[0])



    

# get only valuable values not nan
        #substitute missing values with nan 
        sfc_pressure[ens_memb] = fv.fill_nan(sfc_pressure[ens_memb][:,:])
        mslp[ens_memb] = fv.fill_nan(mslp[ens_memb][:])
        air_temperature_2m[ens_memb] = fv.fill_nan(air_temperature_2m[ens_memb][:,:])
        x_wind_10m[ens_memb] = fv.fill_nan(x_wind_10m[ens_memb][:,:])
        y_wind_10m[ens_memb] = fv.fill_nan(y_wind_10m[ens_memb][:,:])
#### calculate wind direction and speed
# Create wind speed and direction variables
        wd_MEPS[ens_memb] = np.arctan2(y_wind_10m[ens_memb][:,0], x_wind_10m[ens_memb][:,0])
        ws_MEPS[ens_memb] = np.sqrt((x_wind_10m[ens_memb][:,0])**2 + (y_wind_10m[ens_memb][:,0])**2) 

        wd_MEPS[ens_memb] = 270-np.rad2deg(wd_MEPS[ens_memb])  # convert from math to meteo direction and to degrees
        
        
        larger360 = np.where(wd_MEPS[ens_memb] >360.)
        wd_MEPS[ens_memb][larger360] = wd_MEPS[ens_memb][larger360]-360
        
#### save calendar day etc. for scatter plot
    calendar_day.append(calday)
    initialisation_day.append(ini_day)
    calendar_month.append(calmon)
    calendar_year.append(year)
    initialisation_hh.append(hh)
########

#### calculate the difference between obs and model

    ### Wind direction
    diff_wd, Diff_wd_0, Diff_wd_1, Diff_wd_2, Diff_wd_3, Diff_wd_4,\
        Diff_wd_5, Diff_wd_6, Diff_wd_7, \
        Diff_wd_8,Diff_wd_9 = calc_diff(diff_wd,wd_MEPS,WD, Diff_wd_0, Diff_wd_1, \
                                        Diff_wd_2, Diff_wd_3, Diff_wd_4,\
                                        Diff_wd_5, Diff_wd_6, Diff_wd_7, \
                                        Diff_wd_8,Diff_wd_9,'WD')
    
    for ens_memb in range(0,10):
        if (diff_wd[ens_memb] < -180).any() == True:
            diff_wd[ens_memb][np.where(diff_wd[ens_memb] < -180)] = \
                    diff_wd[ens_memb][np.where(diff_wd[ens_memb] < -180)]+360
                
        elif (diff_wd[ens_memb] > 180).any() == True:
            diff_wd[ens_memb][np.where(diff_wd[ens_memb] > 180)] = \
                    diff_wd[ens_memb][np.where(diff_wd[ens_memb] > 180)]-360
        
    
    all_day_max_WD, all_day_min_WD = diff_all_day(diff_wd, all_day_max_WD, all_day_min_WD)
    #####
    
    ### Wind speed
    diff_ws, Diff_ws_0, Diff_ws_1, Diff_ws_2, Diff_ws_3, Diff_ws_4,\
        Diff_ws_5, Diff_ws_6, Diff_ws_7, \
        Diff_ws_8,Diff_ws_9 = calc_diff(diff_ws,ws_MEPS,WS, Diff_ws_0, Diff_ws_1, \
                                        Diff_ws_2, Diff_ws_3, Diff_ws_4,\
                                        Diff_ws_5, Diff_ws_6, Diff_ws_7, \
                                        Diff_ws_8,Diff_ws_9, 'WS')
    
    all_day_max_WS, all_day_min_WS = diff_all_day(diff_ws, all_day_max_WS, all_day_min_WS)
    #####
    
    ### Temperature
    diff_temp, Diff_temp_0, Diff_temp_1, Diff_temp_2, Diff_temp_3, Diff_temp_4,\
        Diff_temp_5, Diff_temp_6, Diff_temp_7, \
        Diff_temp_8,Diff_temp_9 = calc_diff(diff_temp,air_temperature_2m,Temperature, Diff_temp_0, Diff_temp_1, \
                                        Diff_temp_2, Diff_temp_3, Diff_temp_4,\
                                        Diff_temp_5, Diff_temp_6, Diff_temp_7, \
                                        Diff_temp_8,Diff_temp_9,'T2')
    
    all_day_max_temp, all_day_min_temp = diff_all_day(diff_temp, all_day_max_temp, all_day_min_temp)
    #####
    
    ### surface pressure
    diff_mslp, Diff_mslp_0, Diff_mslp_1, Diff_mslp_2, Diff_mslp_3, Diff_mslp_4,\
        Diff_mslp_5, Diff_mslp_6, Diff_mslp_7, \
        Diff_mslp_8,Diff_mslp_9 = calc_diff(diff_mslp,mslp,Pressure, Diff_mslp_0, Diff_mslp_1, \
                                            Diff_mslp_2, Diff_mslp_3, Diff_mslp_4,\
                                            Diff_mslp_5, Diff_mslp_6, Diff_mslp_7, \
                                            Diff_mslp_8,Diff_mslp_9,'SP')
    
    all_day_max_mslp, all_day_min_mslp = diff_all_day(diff_mslp, all_day_max_mslp, all_day_min_mslp)
    #####
    

#########################################

    for ens_memb in range(0,10):
        lead_time_sfc[ens_memb] = lead_time_sfc[ens_memb][~np.isnan(sfc_pressure[ens_memb][:,0])]
        lead_time_2m[ens_memb] = lead_time_2m[ens_memb][~np.isnan(air_temperature_2m[ens_memb][:,0])]
        lead_time_10m[ens_memb] = lead_time_10m[ens_memb][~np.isnan(wd_MEPS[ens_memb])]
        

        sfc_pressure[ens_memb] = sfc_pressure[ens_memb][~np.isnan(sfc_pressure[ens_memb])]
        mslp[ens_memb] = mslp[ens_memb][~np.isnan(mslp[ens_memb])]
        air_temperature_2m[ens_memb] = air_temperature_2m[ens_memb][~np.isnan(air_temperature_2m[ens_memb])]
        ws_MEPS[ens_memb] = ws_MEPS[ens_memb][~np.isnan(wd_MEPS[ens_memb])]
        wd_MEPS[ens_memb] = wd_MEPS[ens_memb][~np.isnan(wd_MEPS[ens_memb])]    

    if hh < 10:
        hh = '0%s' %hh
    else:
        hh = hh

        
    for ens_memb in range(0,10):
        fn_sfc[ens_memb].close()
        fn_2m[ens_memb].close()
        fn_10m[ens_memb].close()

    fig_name = '%s%s%s_%s.png' %(year,month,ini_day,hh)
    
    title = 'initalised: %s, %s %s %s %s UTC' %(calday,ini_day,calmon,year,hh)

####### plot wind direction     
    lgd = plt_variable(lead_time_10m,wd_MEPS,WD,var='WD')
    
    
    if savefig == 1:
        cF.createFolder('%s/wind_direction' %(figdir_10m))
        plt.savefig('%s/wind_direction/%s' % (figdir_10m, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
        print('plot saved: %s/wind_direction/%s' %(figdir_10m,fig_name))
    elif savefig == 0:
        plt.show()
    plt.close()
    
    
    
    
####### plot wind speed    
    lgd = plt_variable(lead_time_10m,ws_MEPS,WS,var='WS')
    
    
    if savefig == 1:
        cF.createFolder('%s/wind_speed' %(figdir_10m))
        plt.savefig('%s/wind_speed/%s' % (figdir_10m, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
        print('plot saved: %s/wind_speed/%s' %(figdir_10m,fig_name))
    elif savefig == 0:
        plt.show()
    plt.close()
    
    

####### plot Temperature 
    lgd = plt_variable(lead_time_2m,air_temperature_2m,Temperature, var = 'T2')
    
    
    if savefig == 1:
        plt.savefig('%s/%s' % (figdir_2m, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
        print('plot saved: %s/%s' %(figdir_2m,fig_name))
    elif savefig == 0:
        plt.show()
    plt.close()

    
####### plot Pressure 
    lgd = plt_variable(lead_time_sfc,mslp,Pressure, var = 'SP')
    
    
    if savefig == 1:
        plt.savefig('%s/%s' % (figdir_sfc, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
        print('plot saved: %s/%s' %(figdir_sfc,fig_name))
    elif savefig == 0:
        plt.show()
    plt.close()
###################################################################    
#### plot difference between observation and ensemble members
    if day == '20':
        cc = no1
    elif day == '21':
        cc = no2
    elif day == '22':
        cc = no3
    elif day == '23':
        cc = no4
    elif day == '24':
        cc = no5
    elif day == '25':
        cc = no6
    elif day == '26':
        cc = no7
    
    plt_scatter_obs_model(WD, wd_MEPS,cc,var='WD')
    if savefig == 1:
        cF.createFolder('%s/wind_direction/' % (figdir_scatter))
        plt.savefig('%s/wind_direction/%s' % (figdir_scatter, fig_name))
        print('plot saved: %s/wind_direction/%s' %(figdir_sfc,fig_name))
    elif savefig == 0:
        plt.show()
    plt.close()
### Wind speed
    plt_scatter_obs_model(WS, ws_MEPS,cc,var='WS')
    if savefig == 1:
        cF.createFolder('%s/wind_speed/' % (figdir_scatter))
        plt.savefig('%s/wind_speed/%s' % (figdir_scatter, fig_name))
        print('plot saved: %s/wind_speed/%s' %(figdir_sfc,fig_name))
    elif savefig == 0:
        plt.show()
    plt.close()
    
### Tempearture    
    plt_scatter_obs_model(Temperature, air_temperature_2m, cc, var='T2')
    if savefig == 1:
        cF.createFolder('%s/%s/' % (figdir_scatter,var_name_2m))
        plt.savefig('%s/%s/%s' % (figdir_scatter,var_name_2m, fig_name))
        print('plot saved: %s/%s/%s' % (figdir_scatter,var_name_2m, fig_name))
    elif savefig == 0:
        plt.show()
    plt.close()
### Pressure    
    plt_scatter_obs_model(Pressure, mslp, cc, var='SP')
    if savefig == 1:
        cF.createFolder('%s/%s/' % (figdir_scatter,var_name_sfc))
        plt.savefig('%s/%s/%s' % (figdir_scatter,var_name_sfc, fig_name))
        print('plot saved: %s/%s/%s' % (figdir_scatter,var_name_sfc, fig_name))
    elif savefig == 0:
        plt.show()
    plt.close()

    
## scatter difference plot

    ### wind direction
    plt_scatter_daily(diff_wd, lead_time_10m, WD, cc, var='WD')
    if savefig == 1:
        cF.createFolder('%s/obs_vs_model/wind_direction/' % (figdir_scatter))
        plt.savefig('%s/obs_vs_model/wind_direction/%s' % (figdir_scatter, fig_name))
        print('plot saved: %s/obs_vs_model/wind_direction/%s' %(figdir_sfc,fig_name))
    elif savefig == 0:
        plt.show()
    plt.close()
        
    ### wind speed
    plt_scatter_daily(diff_ws, lead_time_10m, WS, cc, var='WS')
    if savefig == 1:
        cF.createFolder('%s/obs_vs_model/wind_speed/' % (figdir_scatter))
        plt.savefig('%s/obs_vs_model/wind_speed/%s' % (figdir_scatter, fig_name))
        print('plot saved: %s/obs_vs_model/wind_speed/%s' %(figdir_sfc,fig_name))
    elif savefig == 0:
        plt.show()
    plt.close()
    
    ### temperature
    plt_scatter_daily(diff_temp, lead_time_2m, Temperature, cc,'T2')
    if savefig == 1:
        cF.createFolder('%s/obs_vs_model/%s/' % (figdir_scatter,var_name_2m))
        plt.savefig('%s/obs_vs_model/%s/%s' % (figdir_scatter,var_name_2m, fig_name))
        print('plot saved: %s/obs_vs_model/%s/%s' % (figdir_scatter,var_name_2m, fig_name))
    elif savefig == 0:
        plt.show()
    plt.close()
    
    
    ### pressure
    plt_scatter_daily(diff_mslp, lead_time_sfc, Pressure, cc,'SP')
    if savefig == 1:
        cF.createFolder('%s/obs_vs_model/%s/' % (figdir_scatter,var_name_sfc))
        plt.savefig('%s/obs_vs_model/%s/%s' % (figdir_scatter,var_name_sfc, fig_name))
        print('plot saved: %s/obs_vs_model/%s/%s' % (figdir_scatter,var_name_sfc, fig_name))
    elif savefig == 0:
        plt.show()
    plt.close()

/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/ipykernel_launcher.py:93: RuntimeWarning: invalid value encountered in greater
/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/ipykernel_launcher.py:110: RuntimeWarning: invalid value encountered in less
/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/ipykernel_launcher.py:113: RuntimeWarning: invalid value encountered in greater
/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/ipykernel_launcher.py:111: RuntimeWarning: invalid value encountered in less


plot saved: ../../Figures/MEPS/wind/wind_direction/20161220_00.png
plot saved: ../../Figures/MEPS/wind/wind_speed/20161220_00.png
plot saved: ../../Figures/MEPS/air_temperature_2m/20161220_00.png
plot saved: ../../Figures/MEPS/surface_air_pressure/20161220_00.png
plot saved: ../../Figures/MEPS/surface_air_pressure/wind_direction/20161220_00.png
plot saved: ../../Figures/MEPS/surface_air_pressure/wind_speed/20161220_00.png
plot saved: ../../Figures/MEPS/scatter_MEPS_obs/air_temperature_2m/20161220_00.png
plot saved: ../../Figures/MEPS/scatter_MEPS_obs/surface_air_pressure/20161220_00.png
plot saved: ../../Figures/MEPS/surface_air_pressure/obs_vs_model/wind_direction/20161220_00.png
plot saved: ../../Figures/MEPS/surface_air_pressure/obs_vs_model/wind_speed/20161220_00.png
plot saved: ../../Figures/MEPS/scatter_MEPS_obs/obs_vs_model/air_temperature_2m/20161220_00.png
plot saved: ../../Figures/MEPS/scatter_MEPS_obs/obs_vs_model/surface_air_pressure/20161220_00.png
plot saved: ../../Figure

In [22]:
######## Plot all days in one plot and first three days, then last three days

In [23]:
def plt_all_day_diff(Difference,lead_time_sfc,tot,var,day_range):
    #### Plot difference of all days in scatter plot

    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
# Vertical line to see if too high or too little
    ax.axvline(0, color =spagh.vert_col, linewidth=3)

# Horizontal line to show end of day
    ax.axhline(0,color = spagh.vert_col, linewidth = 3)
    ax.axhline(24,color = spagh.vert_col, linewidth = 3)
    ax.axhline(48,color = spagh.vert_col, linewidth = 3)


    for ens_memb in range(0,1):
        for i in day_range:
            plt.scatter(np.asarray(Difference[ens_memb])[i,:],lead_time_sfc[ens_memb][:np.asarray(tot).shape[0]],
                        color = colors[i],
                    alpha = 0.8, s = 200,
                  label='%s %s %s 0%s UTC' %(initialisation_day[i],
                                        calendar_month[i],calendar_year[i],initialisation_hh[i]) )
    
    for ens_memb in range(1,10):
        for i in day_range:
            plt.scatter(np.asarray(Difference[ens_memb])[i,:],np.arange(0,49),color = colors[i],
                alpha = 0.8, s = 200,label='_nolegend_' )   
            
    ### fine tuning
    lgd = ax.legend(loc='center left',bbox_to_anchor=(1, .75),
         fancybox=True, shadow=True, #ncol=3, 
          fontsize=spagh.label_fs-4)
    lgd.set_title( title= 'initialised:')#, fontsize=spagh.label)
    lgd.get_title().set_fontsize(spagh.label_fs-4)
    plt.setp(lgd.get_texts(), color=spagh.blue)


#lgd = ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.37),
 #         fancybox=True, shadow=True, ncol=3, fontsize=spagh.label_fs)
    frame = lgd.get_frame()
    frame.set_facecolor('white')
    
# yaxis
#a = lead_time_sfc[0][0:48]
    ax.set_ylim(-0.5,49-0.5)
    ax.set_ylabel('forecast time', fontsize=spagh.label_fs)
    ax.set_yticks(np.arange(0,49))
    
    xdays = [0, '', '', 3, '' , '',
        6,'','', 9,'','',12,'','',15,'','',18,'','',
         21, '','',
        24, '','',27, '','', 30, '','',
        33,'','', 36,'','',39,'','',42,'','',45,'','',
        48]
    ax.set_yticklabels(xdays, #rotation = 25, 
                   fontsize = spagh.tick_fs)


    if var == 'WD':
        # Horizontal line to show Wind direction
        ax.axvline(45,color=spagh.vert_col, linewidth= 3)
        ax.axvline(90,color=spagh.vert_col, linewidth= 3)
        ax.axvline(90+45,color=spagh.vert_col, linewidth= 3)
        ax.axvline(180,color=spagh.vert_col, linewidth= 3)
        ax.axvline(-45,color=spagh.vert_col, linewidth= 3)
        ax.axvline(-90,color=spagh.vert_col, linewidth= 3)
        ax.axvline(-90-45,color=spagh.vert_col, linewidth= 3)
        ax.axvline(-180,color=spagh.vert_col, linewidth= 3)
        # xaxis
        ax.set_xlim(-112.5,112.5)
        ax.set_xticks(np.arange(-112.5,135,22.5))  
        ax.set_xticklabels([ '', '-90', '', '-45','','0','','45','','90'], fontsize = spagh.tick_fs)
        # title
        ax.set_title('Wind direction', fontsize=spagh.fontsize) 
        ax.set_xlabel('Difference [$^\circ$]' , fontsize=spagh.label_fs)
    elif var == 'WS':
        # xaxis
        ax.set_xlim(-10,15)
        ax.set_xticks(np.arange(-10,17.5,2.5))
        ax.set_xticklabels([-10,'', -5, '',0, '', 5, '',10,'',15], fontsize=spagh.label_fs)
        # title
        ax.set_title('Wind speed', fontsize=spagh.fontsize) 
        ax.set_xlabel('Difference [m$\,$s$^{-1}$]' , fontsize=spagh.label_fs)
        
    elif var == 'T2':
        # xaxis
        ax.set_xlim(-9,4)
        ax.set_xticks(np.arange(-9,5))
        ax.set_xticklabels([-9, '' , '', -6, '' , '', -3, '' , '', 0, 
                            '' , '', 3, '' , ''], fontsize=spagh.tick_fs)
        # title
        ax.set_title('Air Temperature', fontsize=spagh.fontsize) 
        ax.set_xlabel('Difference [$^\circ$C]' , fontsize=spagh.label_fs)
        
    elif var == 'SP':
        # xaxis
        ax.set_xlim(-8, 8)
        ax.set_xticks(np.arange(-8,9))
        ax.set_xticklabels([-8,'' , -6,'', -4,'',-2,'', 0, '',2,'',4,'',6,'',8], fontsize=spagh.tick_fs)
        # title
        ax.set_title('Sea Level Pressure', fontsize=spagh.fontsize)
        ax.set_xlabel('Difference [hPa]' , fontsize=spagh.label_fs)

# tight layout
    plt.tight_layout()
    return(lgd);



In [24]:
Diff_wd = dict()

Diff_wd[0]=(Diff_wd_0)
Diff_wd[1]=(Diff_wd_1)
Diff_wd[2]=(Diff_wd_2)
Diff_wd[3]=(Diff_wd_3)
Diff_wd[4]=(Diff_wd_4)
Diff_wd[5]=(Diff_wd_5)
Diff_wd[6]=(Diff_wd_6)
Diff_wd[7]=(Diff_wd_7)
Diff_wd[8]=(Diff_wd_8)
Diff_wd[9]=(Diff_wd_9)


lgd = plt_all_day_diff(Diff_wd,lead_time_10m,WD,'WD',np.arange(0,len(Diff_wd[0])-1))
fig_name = '%s%s%s_%s_%s.png' %(calendar_year[0],month,initialisation_day[0],initialisation_day[-1],hh)
if savefig == 1:
    plt.savefig('%s/obs_vs_model/wind_direction/%s' % (figdir_scatter, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
    print('plot saved: %s/obs_vs_model/wind_direction/%s' %(figdir_sfc,fig_name))
elif savefig == 0:
    plt.show()
plt.close()


lgd = plt_all_day_diff(Diff_wd,lead_time_10m,WD,'WD',np.arange(0,4))
fig_name = '%s%s%s_%s_%s.png' %(calendar_year[0],month,initialisation_day[0],initialisation_day[4-1],hh)
if savefig == 1:
    plt.savefig('%s/obs_vs_model/wind_direction/%s' % (figdir_scatter, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
    print('plot saved: %s/obs_vs_model/wind_direction/%s' %(figdir_sfc,fig_name))
elif savefig == 0:
    plt.show()
plt.close()

lgd = plt_all_day_diff(Diff_wd,lead_time_10m,WD,'WD',np.arange(4,7))
fig_name = '%s%s%s_%s_%s.png' %(calendar_year[4],month,initialisation_day[4],initialisation_day[7-1],hh)
if savefig == 1:
    plt.savefig('%s/obs_vs_model/wind_direction/%s' % (figdir_scatter, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
    print('plot saved: %s/obs_vs_model/wind_direction/%s' %(figdir_sfc,fig_name))
elif savefig == 0:
    plt.show()
plt.close()
#plt_diff_cluster(Diff_wd,lead_time_10m,WD,'WD',np.arange(4,7))

plot saved: ../../Figures/MEPS/surface_air_pressure/obs_vs_model/wind_direction/20161220_27_00.png
plot saved: ../../Figures/MEPS/surface_air_pressure/obs_vs_model/wind_direction/20161220_23_00.png
plot saved: ../../Figures/MEPS/surface_air_pressure/obs_vs_model/wind_direction/20161224_26_00.png


In [25]:
Diff_ws = dict()

Diff_ws[0]=(Diff_ws_0)
Diff_ws[1]=(Diff_ws_1)
Diff_ws[2]=(Diff_ws_2)
Diff_ws[3]=(Diff_ws_3)
Diff_ws[4]=(Diff_ws_4)
Diff_ws[5]=(Diff_ws_5)
Diff_ws[6]=(Diff_ws_6)
Diff_ws[7]=(Diff_ws_7)
Diff_ws[8]=(Diff_ws_8)
Diff_ws[9]=(Diff_ws_9)

lgd = plt_all_day_diff(Diff_ws,lead_time_10m,WS,'WS',np.arange(0,len(Diff_ws[0])-1))
fig_name = '%s%s%s_%s_%s.png' %(calendar_year[0],month,initialisation_day[0],initialisation_day[-1],hh)
if savefig == 1:
    plt.savefig('%s/obs_vs_model/wind_speed/%s' % (figdir_scatter, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
    print('plot saved: %s/obs_vs_model/wind_speed/%s' %(figdir_sfc,fig_name))
elif savefig == 0:
    plt.show()
plt.close()

lgd = plt_all_day_diff(Diff_ws,lead_time_10m,WS,'WS',np.arange(0,4))
fig_name = '%s%s%s_%s_%s.png' %(calendar_year[0],month,initialisation_day[0],initialisation_day[4-1],hh)
if savefig == 1:
    plt.savefig('%s/obs_vs_model/wind_speed/%s' % (figdir_scatter, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
    print('plot saved: %s/obs_vs_model/wind_speed/%s' %(figdir_sfc,fig_name))
elif savefig == 0:
    plt.show()
plt.close()

lgd = plt_all_day_diff(Diff_ws,lead_time_10m,WS,'WS',np.arange(4,7))
fig_name = '%s%s%s_%s_%s.png' %(calendar_year[4],month,initialisation_day[4],initialisation_day[7-1],hh)
if savefig == 1:
    plt.savefig('%s/obs_vs_model/wind_speed/%s' % (figdir_scatter, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
    print('plot saved: %s/obs_vs_model/wind_speed/%s' %(figdir_sfc,fig_name))
elif savefig == 0:
    plt.show()
plt.close()

plot saved: ../../Figures/MEPS/surface_air_pressure/obs_vs_model/wind_speed/20161220_27_00.png
plot saved: ../../Figures/MEPS/surface_air_pressure/obs_vs_model/wind_speed/20161220_23_00.png
plot saved: ../../Figures/MEPS/surface_air_pressure/obs_vs_model/wind_speed/20161224_26_00.png


In [26]:
Diff_temp = dict()

Diff_temp[0]=(Diff_temp_0)
Diff_temp[1]=(Diff_temp_1)
Diff_temp[2]=(Diff_temp_2)
Diff_temp[3]=(Diff_temp_3)
Diff_temp[4]=(Diff_temp_4)
Diff_temp[5]=(Diff_temp_5)
Diff_temp[6]=(Diff_temp_6)
Diff_temp[7]=(Diff_temp_7)
Diff_temp[8]=(Diff_temp_8)
Diff_temp[9]=(Diff_temp_9)

lgd = plt_all_day_diff(Diff_temp,lead_time_2m,Temperature,'T2',np.arange(0,len(Diff_temp[0])-1))
fig_name = '%s%s%s_%s_%s.png' %(calendar_year[0],month,initialisation_day[0],initialisation_day[-1],hh)
if savefig == 1:
    plt.savefig('%s/obs_vs_model/%s/%s' % (figdir_scatter,var_name_2m, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
    print('plot saved: %s/obs_vs_model/%s/%s' % (figdir_scatter,var_name_2m, fig_name))
elif savefig == 0:
    plt.show()
plt.close()

lgd = plt_all_day_diff(Diff_temp,lead_time_2m,Temperature,'T2',np.arange(0,4))
fig_name = '%s%s%s_%s_%s.png' %(calendar_year[0],month,initialisation_day[0],initialisation_day[4-1],hh)
if savefig == 1:
    plt.savefig('%s/obs_vs_model/%s/%s' % (figdir_scatter,var_name_2m, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
    print('plot saved: %s/obs_vs_model/%s/%s' % (figdir_scatter,var_name_2m, fig_name))
elif savefig == 0:
    plt.show()
plt.close()

lgd = plt_all_day_diff(Diff_temp,lead_time_2m,Temperature,'T2',np.arange(0,len(Diff_temp[0])-1))
fig_name = '%s%s%s_%s_%s.png' %(calendar_year[4],month,initialisation_day[4],initialisation_day[7-1],hh)
if savefig == 1:
    plt.savefig('%s/obs_vs_model/%s/%s' % (figdir_scatter,var_name_2m, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
    print('plot saved: %s/obs_vs_model/%s/%s' % (figdir_scatter,var_name_2m, fig_name))
elif savefig == 0:
    plt.show()
plt.close()

plot saved: ../../Figures/MEPS/scatter_MEPS_obs/obs_vs_model/air_temperature_2m/20161220_27_00.png
plot saved: ../../Figures/MEPS/scatter_MEPS_obs/obs_vs_model/air_temperature_2m/20161220_23_00.png
plot saved: ../../Figures/MEPS/scatter_MEPS_obs/obs_vs_model/air_temperature_2m/20161224_26_00.png


In [27]:
Diff_mslp = dict()

Diff_mslp[0]=(Diff_mslp_0)
Diff_mslp[1]=(Diff_mslp_1)
Diff_mslp[2]=(Diff_mslp_2)
Diff_mslp[3]=(Diff_mslp_3)
Diff_mslp[4]=(Diff_mslp_4)
Diff_mslp[5]=(Diff_mslp_5)
Diff_mslp[6]=(Diff_mslp_6)
Diff_mslp[7]=(Diff_mslp_7)
Diff_mslp[8]=(Diff_mslp_8)
Diff_mslp[9]=(Diff_mslp_9)

lgd = plt_all_day_diff(Diff_mslp,lead_time_sfc,Pressure,'SP',np.arange(0,len(Diff_mslp[0])-1))
fig_name = '%s%s%s_%s_%s.png' %(calendar_year[0],month,initialisation_day[0],initialisation_day[-1],hh)
if savefig == 1:
    plt.savefig('%s/obs_vs_model/%s/%s' % (figdir_scatter,var_name_sfc, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
    print('plot saved: %s/obs_vs_model/%s/%s' % (figdir_scatter,var_name_sfc, fig_name))
elif savefig == 0:
    plt.show()
plt.close()



lgd = plt_all_day_diff(Diff_mslp,lead_time_sfc,Pressure,'SP',np.arange(0,4))
fig_name = '%s%s%s_%s_%s.png' %(calendar_year[0],month,initialisation_day[0],initialisation_day[4-1],hh)
if savefig == 1:
    plt.savefig('%s/obs_vs_model/%s/%s' % (figdir_scatter,var_name_sfc, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
    print('plot saved: %s/obs_vs_model/%s/%s' % (figdir_scatter,var_name_sfc, fig_name))
elif savefig == 0:
    plt.show()
plt.close()

lgd = plt_all_day_diff(Diff_mslp,lead_time_sfc,Pressure,'SP',np.arange(4,7))
fig_name = '%s%s%s_%s_%s.png' %(calendar_year[4],month,initialisation_day[4],initialisation_day[7-1],hh)
if savefig == 1:
    plt.savefig('%s/obs_vs_model/%s/%s' % (figdir_scatter,var_name_sfc, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
    print('plot saved: %s/obs_vs_model/%s/%s' % (figdir_scatter,var_name_sfc, fig_name))
elif savefig == 0:
    plt.show()
plt.close()

plot saved: ../../Figures/MEPS/scatter_MEPS_obs/obs_vs_model/surface_air_pressure/20161220_27_00.png
plot saved: ../../Figures/MEPS/scatter_MEPS_obs/obs_vs_model/surface_air_pressure/20161220_23_00.png
plot saved: ../../Figures/MEPS/scatter_MEPS_obs/obs_vs_model/surface_air_pressure/20161224_26_00.png
